In [93]:
import matplotlib.pyplot as plt
import glob
from PIL import Image, ImageDraw, ImageFont
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit

from skimage import feature, data, color
from glob import glob # Used to easily find file paths
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [94]:
# Loud the data folder and the images
col_names = ['REFNUM', 'BG', 'CLASS', 'SEVERITY', 'X', 'Y', 'RADIUS']
df = pd.read_csv('data2.txt', sep="\s+", names=col_names, header=None)
df['CANCER'] = df['SEVERITY'].apply(lambda x: 1 if x in ['B', 'M'] else 0)

images_path = "all-mias"

all_images = []
all_labels = []

for filename in sorted(os.listdir(images_path)):
    if filename.lower().endswith('.pgm'):
        ref_num = os.path.splitext(filename)[0]
        record = df[df['REFNUM'] == ref_num]

        if not record.empty:
            full_path = os.path.join(images_path, filename)
            img_array = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)

            labels = record['CANCER'].iloc[0]

            all_images.append(img_array)
            all_labels.append(labels)

print("Besham ta alan ke chedesh ...")

Besham ta alan ke chedesh ...


In [95]:
print(f"Image list {len(all_images)}")
print(f"Labels list {len(all_labels)}")
df.head(5)

Image list 324
Labels list 324


,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS,CANCER
0,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS,0
1,mdb001,G,CIRC,B,535,425,197,1
2,mdb002,G,CIRC,B,522,280,69,1
3,mdb003,D,NORM,NaN,NaN,NaN,NaN,0
4,mdb004,D,NORM,NaN,NaN,NaN,NaN,0


In [113]:
# img = cv2.imread('all-mias/mdb005.pgm', cv2.IMREAD_GRAYSCALE)
# print(img.shape, img.dtype)
# img.max()
# plt.imshow(img, cmap='gray')
normalaized_img = []
print("Computing the image and histogram for pooling and classification...")

for img in all_images:
    # Normalazing the image 
    if img.max() > img.min():
        n_img = ((img - img.min()) / (img.max() - img.min()) * 255) 
    else:
        n_img = img
    # normalaized_img.append(n_img)
    P = 8
    R = 1
    method = 'uniform'
    LBP = feature.local_binary_pattern(n_img, P, R, method)

    bins_num = int(n_img.max()) + 1
    normalaized_img.append(bins_num)
# print(LBP.min())

print("Normalazation process has been Done!.")

print("Start of the histogram processing...")
def compute_histograms_for_each_region(normalaized_img, G=6):
    #compute th histogram from 255 in image size to 59 =>  reduce the number of bins
    histogram = []
    # Pooling the image in to 6 x 6
    H = 1020 # height
    W = 1020 # width
    #G = 6 # Number of Grid 6 * 6 
    hor = int(H / G) # height of each region
    wor = int(W / G) # width of each region

# Pooling the data into 6 * 6 regions
    for i in range(G):
        for j in range(G):
            row_start = i * hor
            row_end = (i + 1) * hor
            col_start = j * wor
            col_end = (i + 1) * wor
            region = img[row_start:row_end, col_start:col_end]

        hist, _ = np.histogram(n_img.ravel(), bins=bins_num, range=(0, bins_num))
        histogram.append(hist)

    return histogram
x_list = []
y_list = []
for img, label in zip(all_images, all_labels):
    histogram = compute_histograms_for_each_region(img)

    f_vector = np.concatenate(histogram)

    x_list.append(f_vector)
    y_list.append(label)

X = np.array(x_list)
y = np.array(y_list)

# print(hor)

print("Function has been done successfully")

andaki sabr sahar nazdik ast...
Normalazation process has been Done!.
Start of the histogram processing...


[[383114     38   7288 ...    100      0      8]
 [383114     38   7288 ...    100      0      8]
 [383114     38   7288 ...    100      0      8]
 ...
 [383114     38   7288 ...    100      0      8]
 [383114     38   7288 ...    100      0      8]
 [383114     38   7288 ...    100      0      8]]


In [98]:
unique_codes = np.unique(LBP)
print("Unique LBP codes in this image:", unique_codes)

Unique LBP codes in this image: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [114]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,    # for reproducibility
    stratify=y          # preserves class ratios
)

In [115]:
rf_model = RandomForestClassifier(
    n_estimators=1500,
    max_depth=None,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# 5. Predict
y_pred = rf_model.predict(X_test)
y_prob = rf_model.predict_proba(X_test)  # if you need probabilities


In [116]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      1.00      0.79        42
           1       0.00      0.00      0.00        23

    accuracy                           0.65        65
   macro avg       0.32      0.50      0.39        65
weighted avg       0.42      0.65      0.51        65

Confusion Matrix:
[[42  0]
 [23  0]]


C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [107]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import StandardScaler
from sklearn.svm            import SVC
from sklearn.metrics        import classification_report, confusion_matrix, roc_auc_score

# 1. Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 2. Scale features (fit on train, apply to both)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 3. Instantiate SVM
svm = SVC(
    kernel='rbf',    # radial basis function
    C=1.0,           # regularization parameter
    gamma='scale',   # 1 / (n_features * X.var())
    probability=True,# if you need predict_proba()
    random_state=42
)

# 4. Train
svm.fit(X_train_scaled, y_train)

# 5. Predict
y_pred = svm.predict(X_test_scaled)
y_prob = svm.predict_proba(X_test_scaled)[:, 1]  # for binary or one‑vs‑rest

# 6. Evaluate
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
try:
    print("ROC AUC:", roc_auc_score(y_test, y_prob, multi_class='ovr'))
except:
    pass

# 7. (Optional) Hyperparameter tuning
param_grid = {
    'C':     [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}
grid = GridSearchCV(
    svm, param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)
grid.fit(X_train_scaled, y_train)
print("Best SVM params:", grid.best_params_)
print("Best CV score:", grid.best_score_)

# After tuning, you can re‑evaluate the best model:
best_svm = grid.best_estimator_
y_pred_best = best_svm.predict(X_test_scaled)
print("Tuned SVM report:\n", classification_report(y_test, y_pred_best))


C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.65      1.00      0.79        42
           1       0.00      0.00      0.00        23

    accuracy                           0.65        65
   macro avg       0.32      0.50      0.39        65
weighted avg       0.42      0.65      0.51        65

Confusion Matrix:
 [[42  0]
 [23  0]]
ROC AUC: 0.5
Best SVM params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Best CV score: 0.49588881706528765
Tuned SVM report:
               precision    recall  f1-score   support

           0       0.65      1.00      0.79        42
           1       0.00      0.00      0.00        23

    accuracy                           0.65        65
   macro avg       0.32      0.50      0.39        65
weighted avg       0.42      0.65      0.51        65



C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [82]:
# plt.figure(figsize=(12,4))
# plt.plot(n_img)
# plt.title("LBP Histogram")
# plt.xlabel("LBP code")
# plt.ylabel("Frequency")
# plt.grid()
# plt.show()

In [105]:
# fig, axes = plt.subplots(1, 2, figsize=(8, 4))
# axes[0].imshow(n_img, cmap='gray')
# axes[0].set_title("Original")
# axes[0].axis('off')

# axes[1].imshow(LBP, cmap='gray')
# axes[1].set_title(f"LBP (P={P}, R={R})")
# axes[1].axis('off')
# plt.tight_layout()
# plt.show()

In [117]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# --- 1. Load Data (Your existing code, which is correct) ---
print("Loading and preparing data...")
col_names = ['REFNUM', 'BG', 'CLASS', 'SEVERITY', 'X', 'Y', 'RADIUS']
df = pd.read_csv('data2.txt', sep="\s+", names=col_names, header=None)
df['CANCER'] = df['SEVERITY'].apply(lambda x: 1 if x in ['B', 'M'] else 0)

images_path = "all-mias"
all_images = []
all_labels = []

for filename in sorted(os.listdir(images_path)):
    if filename.lower().endswith('.pgm'):
        ref_num = os.path.splitext(filename)[0]
        record = df[df['REFNUM'] == ref_num]
        if not record.empty:
            full_path = os.path.join(images_path, filename)
            img_array = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
            labels = record['CANCER'].iloc[0]
            all_images.append(img_array)
            all_labels.append(labels)
print("Data loading complete.")

# --- 2. Corrected LBP Feature Extraction ---
print("Starting feature extraction with LBP...")

def create_lbp_feature_vector(image, P=8, R=3, grid_cells=6):
    """
    Computes the Local Binary Pattern for an image, divides it into a grid,
    and returns a concatenated histogram of all grid cells.
    
    Args:
        image (np.array): The input grayscale image.
        P (int): Number of circularly symmetric neighbor set points.
        R (int): Radius of circle.
        grid_cells (int): The number of cells in one dimension (e.g., 6 for a 6x6 grid).

    Returns:
        np.array: A single, flattened feature vector for the image.
    """
    # 1. Compute the LBP image
    # The 'uniform' method is robust and creates 10 bins (P+2)
    lbp_image = feature.local_binary_pattern(image, P, R, method='uniform')
    
    # 2. Calculate histogram properties
    # For 'uniform' LBP, the number of bins is P + 2.
    num_bins = int(lbp_image.max() + 1)
    
    # 3. Divide the LBP image into a grid and compute histograms
    h, w = lbp_image.shape
    cell_h, cell_w = h // grid_cells, w // grid_cells
    
    full_histogram = []
    for i in range(grid_cells):
        for j in range(grid_cells):
            # Extract the region from the LBP image
            region = lbp_image[i*cell_h:(i+1)*cell_h, j*cell_w:(j+1)*cell_w]
            
            # Compute the histogram for this region
            # The range is from 0 to num_bins.
            hist, _ = np.histogram(region, bins=num_bins, range=(0, num_bins))
            
            # Add the region's histogram to our list
            full_histogram.append(hist)
            
    # 4. Concatenate all regional histograms into a single feature vector
    return np.concatenate(full_histogram)

# Now, create the feature matrix (X) and label vector (y)
X_features = []
y_labels = []

for img, label in zip(all_images, all_labels):
    # For each image, compute its LBP feature vector
    lbp_features = create_lbp_feature_vector(img)
    X_features.append(lbp_features)
    y_labels.append(label)

# Convert lists to NumPy arrays for scikit-learn
X = np.array(X_features)
y = np.array(y_labels)

print("Feature extraction complete.")
print(f"Shape of our feature matrix X: {X.shape}")
print(f"Shape of our label vector y: {y.shape}")


# --- 3. Train and Evaluate the Random Forest (Your existing code, now with good features) ---
print("\nSplitting data and training Random Forest model...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

rf_model = RandomForestClassifier(
    n_estimators=150,
    max_depth=None,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

print("\n--- Model Evaluation Results ---")
print(classification_report(y_test, y_pred, target_names=['Normal (0)', 'Cancer (1)']))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Loading and preparing data...
Data loading complete.
Starting feature extraction with LBP...
Feature extraction complete.
Shape of our feature matrix X: (324, 360)
Shape of our label vector y: (324,)

Splitting data and training Random Forest model...

--- Model Evaluation Results ---
              precision    recall  f1-score   support

  Normal (0)       0.60      0.79      0.68        42
  Cancer (1)       0.10      0.04      0.06        23

    accuracy                           0.52        65
   macro avg       0.35      0.41      0.37        65
weighted avg       0.42      0.52      0.46        65

Confusion Matrix:
[[33  9]
 [22  1]]
